In [1]:
%load_ext autoreload
%autoreload 2
%cd '/home/xingyu/Projects/PlasticineLab'
%pwd
%matplotlib inline
import matplotlib.pyplot as plt
import taichi as ti
import numpy as np
import cv2
import os
from plb.utils.visualization_utils import save_rgb

target_path = './datasets/0907_PushSpread/init'
if not os.path.exists(target_path):
    os.makedirs(target_path, exist_ok=True)

[Errno 2] No such file or directory: '/home/xingyu/Projects/PlasticineLab'
/home/jianrenw/carl/research/dev/PlasticineLab/imitation/gen_data
[Taichi] mode=release
[Taichi] preparing sandbox at /tmp/taichi-zy6oelot
[Taichi] version 0.7.26, llvm 10.0.0, commit e37bdb5e, linux, python 3.8.11
[I 10/20/21 15:21:58.646 187953] [shell.py:_shell_pop_print@35] Graphical python shell detected, using wrapped sys.stdout


In [3]:
import lzma
import pickle
import numpy as np
import os
np.random.seed(0)
def generate_target(env_name):
    from plb.envs.multitask_env import MultitaskPlasticineEnv
    env = MultitaskPlasticineEnv(cfg_path=f'{env_name}_target.yml', generating_cached_state=True)
    taichi_env = env.taichi_env

    if env_name == 'push_spread':
        N = 1
        xs = np.linspace(0.4, 0.55, N)
        
        def randomize_tool_state(cfg, box_min_x, pin_min_y):
            # init_pos: (0.7, 0.10, 0.5)
            pos = eval(cfg.PRIMITIVES[1]['init_pos'])
            box_dist = np.random.rand() * 0.1 + 0.05
            cfg.PRIMITIVES[1]['init_pos'] = (float(box_min_x + box_dist), pos[1], pos[2])
            
            pos = eval(cfg.PRIMITIVES[0]['init_pos'])
            height = np.random.rand() * 0.05
            x_shift = np.random.rand() * 0.2 - 0.15
            cfg.PRIMITIVES[0]['init_pos'] = (pos[0] + x_shift, float(pin_min_y + height), pos[2])
            
            
        def case1(cfg): # Sphere
            rs = np.linspace(0.06, 0.09, N)
            # print('setting x pos {}, radius {}'.format(xs[i], rs[j]))
            
            pos = eval(cfg.SHAPES[0]['init_pos'])
            cfg.SHAPES[0]['shape'] = 'sphere'
            del cfg.SHAPES[0]['width']
            cfg.SHAPES[0]['radius'] = rs[j]
            new_pos = (xs[i], rs[j] + 0.01, pos[2])
            cfg.SHAPES[0]['init_pos'] = new_pos
            
            randomize_tool_state(cfg, xs[i] + rs[j], 0.19)
        
        def case2(cfg): # Capsule
            rs = np.linspace(0.04, 0.06, N)
            capsule_h = 0.2
            # print('setting x pos {}, radius {}, height {}'.format(xs[i], rs[j], capsule_h))
            
            pos = eval(cfg.SHAPES[0]['init_pos'])
            cfg.SHAPES[0]['shape'] = 'capsule'
            del cfg.SHAPES[0]['width']
            cfg.SHAPES[0]['radius'] = rs[j]
            cfg.SHAPES[0]['height'] = capsule_h
            new_pos = (xs[i], rs[j] + 0.01, pos[2])
            cfg.SHAPES[0]['init_pos'] = new_pos
            
            randomize_tool_state(cfg, xs[i] + rs[j], 0.19)
            
        def case3(cfg):
            w = np.linspace(0.15, 0.25, N)
            h = np.linspace(0.03, 0.06, N)
            w_id, h_id = np.random.randint(0, N),  np.random.randint(0, N)
            pos = eval(cfg.SHAPES[0]['init_pos'])
            size = eval(cfg.SHAPES[0]['width'])
            new_pos = (xs[i], h[h_id], pos[2])
            new_size = (w[w_id], h[h_id], size[2])
            cfg.SHAPES[0]['init_pos'] = new_pos
            cfg.SHAPES[0]['width'] = new_size
            randomize_tool_state(cfg, xs[i] + w[w_id], 0.19)
            
        target_id =0 
        target_imgs = []
        all_imgs = []
        for case in [case1, case2, case3]:
            for i in range(N):
                for j in range(N):
                    env.reset(target_cfg_modifier=case)
                    img = taichi_env.render(mode='rgb', target=False, shape=1, primitive=1)
                    all_imgs.append(img[:, :, :3])
                    state = taichi_env.get_state()
                    state_name = os.path.join(target_path, f'state_{target_id}.xz')
                    with lzma.open(state_name, 'wb') as f:
                        pickle.dump(state, f, protocol=4)
                    # taichi_env.simulator.clear_and_compute_grid_m(0)
                    # np.save(os.path.join(target_path, f'target_{target_id}.npy'), taichi_env.simulator.get_m())
                    save_rgb(os.path.join(target_path, f'state_{target_id}.png'), np.array(img[:, :, :3]).astype(np.float32))
                    target_id += 1
                    target_imgs.append(img)
        plt.Figure()
        plt.imshow(np.max(np.array(all_imgs), axis=0))                                                                                                           
        plt.show()
        # def case2(cfg):
        #     pos = eval(cfg.SHAPES[0]['init_pos'])
        #     cfg.SHAPES[0]['shape'] = 'sphere'
        #     del cfg.SHAPES[0]['width']
        #     cfg.SHAPES[0]['radius'] = 0.07
        #     new_pos = (xs[i], 0.08, pos[2])
        #     cfg.SHAPES[0]['init_pos'] = new_pos
        # 
        # for i in range(10):
        #     env.reset(target_cfg_modifier=case2)
        #     img = taichi_env.render(mode='rgb', target=False, shape=1, primitive=1)
        #     taichi_env.simulator.clear_and_compute_grid_m(0)
        #     np.save(os.path.join(target_path, f'target_{i + 10}.npy'), taichi_env.simulator.get_m())
        #     save_rgb(os.path.join(target_path, 'target_{}.png'.format(i + 10)), img)
        #     target_imgs.append(img)
        # 
        # np.save(os.path.join(target_path, 'target_imgs.npy'), np.array(target_imgs))
    else:
        raise NotImplementedError


generate_target('push_spread')



pimirives: num primitive: 2
Building primitive
action:
  dim: 6
  scale: (0.7, 0.005, 0.005, 0.005, 0.0, 0.0)
collision_group: [0.0, 0.0, 0.0]
color: (0.7568, 0.6039, 0.4196)
friction: 0.9
h: 0.3
init_pos: (0.55, 0.2, 0.5)
init_rot: (0.707, 0.707, 0.0, 0.0)
lower_bound: (0.0, 0.09, 0.0)
r: 0.03
shape: RollingPinExt
upper_bound: (1.0, 1.0, 1.0)
variations: None
Building primitive
action:
  dim: 6
  scale: (0.015, 0.0, 0.0, 0.0, 0.0, 0.0)
collision_group: [0.0, 0.0, 0.0]
color: (0.7568, 0.6039, 0.4196)
friction: 0.9
init_pos: (0.7, 0.1, 0.5)
init_rot: (0.707, 0.707, 0.0, 0.0)
lower_bound: (0.0, 0.0, 0.0)
shape: Box
size: (0.015, 0.15, 0.1)
upper_bound: (1.0, 1.0, 1.0)
variations: None
Initialize Tina Renderer
bake_size: 6  
cam_center: (0.55, 0.07, 0.5)  
cam_phi: -0.5  
cam_radius: 0.75  
cam_theta: 0.0  
camera_pos: (0.5, 1.2, 4.0)  
camera_rot: (0.2, 0)  
dx: 0.006666666666666667  
image_res: (128, 128)  
light_direction: (0.0, 1.0, 1.0)  
max_num_particles: 1000000  
max_ray_depth: 2

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType